# Tools Agent

# Search Engine With Tools And Agents


In [36]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [37]:
# use langchain inbuilt tools

api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

wikipedia


In [38]:
api_wrapper_axriv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_axriv)
print(arxiv.name)

arxiv


In [39]:
tools=[wiki,arxiv]

In [40]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter,TextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [42]:
loader=WebBaseLoader("https://www.investopedia.com/how-nvidia-makes-money-4799532")


In [43]:
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
documents=text_splitter.split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retrievar=vectordb.as_retriever()
retrievar

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1586a78c0>, search_kwargs={})

In [44]:
from langchain.tools.retriever import create_retriever_tool
retrievar_tool=create_retriever_tool(retrievar,"nvidia-expert","What is Nvidia income source")
print(retrievar_tool.name)

nvidia-expert


In [45]:
tools=[wiki,arxiv,retrievar_tool]

In [46]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/rahulkumar/Documents/GitHub/LLMs_Agent/Langchain_Langgraph/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='nvidia-expert', description='What is Nvidia income source', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x11c44ff60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0

​The hwchase17/openai-functions-agent is a pre-defined agent prompt available in the LangChain Hub, designed to facilitate the integration of OpenAI's function-calling capabilities within LangChain applications. This agent enables language models, such as GPT-3.5 or GPT-4, to intelligently decide when and which external functions (tools) to invoke based on user input.​


In [47]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama3-8b-8192")

### Prompt template

In [48]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [49]:
## Agents

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [50]:
# Agent Executer

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [52]:
response=agent_executor.invoke({"input":"How India got independence?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'How India got independence'}`


Page: Independence Day (India)
Summary: Independence Day is celebrated annually on 15 August as a public holiday in India commemorating the nation's independence from the United Kingdom on 15 August 1947. On this day the Indian Independence Act 1947 
Invoking: `wikipedia` with `{'query': 'Indian Independence Act 1947'}`


Page: Indian Independence Act 1947
Summary: The Indian Independence Act 1947 (10 & 11 Geo. 6. c. 30) is an act of the Parliament of the United Kingdom that partitioned British India into the two new independent dominions of India and Pakistan. The Ac
Invoking: `arxiv` with `{'query': 'partitioning british india'}`


Published: 2023-04-16
Title: Study on the tea market in India
Authors: Adit Vinod Nair, Adarsh Damani, Devansh Khandelwal, Harshita Sachdev, Sreayans Jain
Summary: India's tea business has a long history and plays a significant role in the
econom

In [53]:
print(response)

{'input': 'How India got independence?', 'output': "I think it's time to respond directly without using a tool!\n\nThe Indian Independence Act 1947 partitioned British India into the two new independent dominions of India and Pakistan. The partition was a result of the negotiations between the British government and the Indian National Congress, led by Jawaharlal Nehru and Muhammad Ali Jinnah. The Act came into effect on August 15, 1947, and India gained its independence from British colonial rule."}
